# Chapter 5

## SQL: Queries, Programming, Triggers

For this lecture, we will be using this Jupyter notebook to demonstrate queries.  We require the *ipython-sql* module, and the following lines load the module and create an in-memory database:

In [ ]:
%load_ext sql
%sql sqlite://

# Table Creation 

Now let's create some tables for sailors example and then add some data to them.  

We will create a table for:
* sailors
* boats
* reservations

*Note: I have added in a command to DROP the tables if they already exist.  This makes it easier to re-run these commands if necessary.*

In [ ]:
%%sql 

DROP TABLE IF EXISTS sailors; 

CREATE TABLE sailors ( 
    sailor_id INTEGER PRIMARY KEY, 
    sailor_name varchar(20), 
    rating int, 
    age float);

In [ ]:
%%sql 

DROP TABLE IF EXISTS boats;

CREATE TABLE boats (
    boat_id INTEGER PRIMARY KEY,
    color VARCHAR(20)
);

In [ ]:
%%sql 
INSERT INTO BOATS (boat_id, color)
VALUES ( 101, 'red' );
INSERT INTO BOATS (boat_id, color)
VALUES ( 103, 'green');

In [ ]:
%%sql 

DROP TABLE IF EXISTS reservations;

CREATE TABLE reservations (
    sailor_id INTEGER, 
    boat_id INTEGER, 
    date int, 
    FOREIGN KEY(sailor_id) REFERENCES sailors(sailor_id));

In [ ]:
%%sql 
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES ( 22, 'Dustin', 7, 45.0 );
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES ( 31, 'Lubber', 8, 55.5 );
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES ( 58, 'Rusty', 10, 35.0 );
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES ( 28, 'Yuppy', 9, 45.0 );
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES ( 48, 'Guppy', 5, 35.0 );

In [ ]:
%%sql
DELETE FROM reservations;


INSERT INTO RESERVATIONS (sailor_id, boat_id, date)
VALUES ( 22, 101, date('1996-10-10') );
INSERT INTO RESERVATIONS (sailor_id, boat_id, date)
VALUES ( 58, 103, date('1996-11-12') );
INSERT INTO RESERVATIONS (sailor_id, boat_id, date)
VALUES ( 58, 101, date('1996-11-15') );



# Review the Data
Let's take a look at all the data we just created:

In [ ]:
%sql SELECT * from sailors;

In [ ]:
%sql SELECT * from boats;

In [ ]:
%sql SELECT * from reservations;

# Example of Conceptual Evaluation

In [ ]:
%%sql

SELECT s.sailor_name 
FROM sailors s, reservations r
WHERE s.sailor_id=r.sailor_id AND r.boat_id=103;

Notice that the following queries return the same results, despite not having or not using the range variables *s* and *r*:

In [ ]:
%%sql 

SELECT s.sailor_name 
FROM sailors s, reservations r
WHERE s.sailor_id=r.sailor_id AND boat_id=103;

In [ ]:
%%sql 

SELECT sailor_name from sailors, reservations
WHERE sailors.sailor_id=reservations.sailor_id AND boat_id=103;

# Find sailors who've reserved at least one boat

In [ ]:
%%sql 

SELECT DISTINCT s.sailor_name, s.sailor_id
FROM sailors s, reservations r
WHERE s.sailor_id=r.sailor_id;

Notice how the query below uses arithmetical expressions in its result, and pattern matching with the *LIKE* keyword:

In [ ]:
%%sql 

SELECT s.sailor_name, s.age, S.age-5 AS age1, 2*s.age AS age2
FROM sailors s
WHERE s.sailor_name LIKE 'Y_%Y';

# Find sailor_ids for sailors who've reserve a red *or* a green boat

In [ ]:
%%sql 

SELECT s.sailor_id
FROM sailors s, boats b, reservations r
WHERE s.sailor_id=r.sailor_id 
    AND r.boat_id=b.boat_id 
    AND (B.color='red' OR b.color='green');

# What if we replace the OR with AND?

In [ ]:
%%sql 

SELECT s.sailor_id
FROM sailors s, boats b, reservations r
WHERE s.sailor_id=r.sailor_id 
    AND r.boat_id=b.boat_id 
    AND (b.color='red' AND b.color='green');

This query probably isn't doing what we wanted it to.  It is looking for boats that are both red and green!  

Another way to get these same results as the original *OR* query is to do a union of two queries.

*Note: they must be union-compatible.*

In [ ]:
%%sql 

SELECT s.sailor_id
FROM sailors s, boats b, reservations r
WHERE s.sailor_id=r.sailor_id 
    AND r.boat_id=b.boat_id 
    AND (B.color='red')

UNION

SELECT s.sailor_id
FROM sailors s, boats b, reservations r
WHERE s.sailor_id=r.sailor_id 
    AND r.boat_id=b.boat_id 
    AND (B.color='green')
    ;

In the same place as the UNION keyword, we could use the EXCEPT keyword.

In [ ]:
%%sql 

SELECT s.sailor_id
FROM sailors s, boats b, reservations r
WHERE s.sailor_id=r.sailor_id 
    AND r.boat_id=b.boat_id 
    AND (B.color='red')

EXCEPT

SELECT s.sailor_id
FROM sailors s, boats b, reservations r
WHERE s.sailor_id=r.sailor_id 
    AND r.boat_id=b.boat_id 
    AND (B.color='green')
    ;

# Find the sailor_ids of sailors who've reserved a red *and* a green boat

In [ ]:
%%sql 

SELECT s.sailor_id 
FROM sailors s, boats b1, reservations r1, boats b2, reservations r2
WHERE s.sailor_id=r1.sailor_id 
    AND r1.boat_id=b1.boat_id
    AND s.sailor_id=r2.sailor_id
    AND r2.boat_id=b2.boat_id
    AND (b1.color='red' AND b2.color='green');

## We can implement the same query by taking the intersection of two queries.

In [ ]:
%%sql 

SELECT s.sailor_id
FROM sailors s, boats b, reservations r
WHERE s.sailor_id=r.sailor_id 
    AND r.boat_id=b.boat_id 
    AND (B.color='red')

INTERSECT

SELECT s.sailor_id
FROM sailors s, boats b, reservations r
WHERE s.sailor_id=r.sailor_id 
    AND r.boat_id=b.boat_id 
    AND (B.color='green')

# Nested Queries

In [ ]:
%%sql 

SELECT s.sailor_name
FROM sailors s
where s.sailor_id IN (
        SELECT r.sailor_id
        FROM reservations r
        WHERE r.boat_id=103)

In [ ]:
%%sql 

SELECT s.sailor_name
FROM sailors s
WHERE EXISTS (
        SELECT *
        FROM reservations r
        WHERE r.boat_id=103
            AND s.sailor_id=r.sailor_id);

# Set Comparison Operators

Our slides make use of the keyword *ANY*.  SQLite does not implement this keyword, but we can create a similar outcome by using the *MIN* function. 

In [ ]:
%%sql 

SELECT * 
FROM sailors s
WHERE s.rating > (
    SELECT MIN(s2.rating) 
    FROM sailors s2
    WHERE s2.sailor_name='Dustin');

# Rewriting INSERSECT Queries Using IN

In [ ]:
%%sql 

SELECT s.sailor_id
FROM sailors s, boats b, reservations r
WHERE s.sailor_id=r.sailor_id 
    AND r.boat_id=b.boat_id
    AND b.color='red'
    AND s.sailor_id IN (
        SELECT S2.sailor_id
        FROM sailors s2, boats b2, reservations r2
        WHERE s2.sailor_id=r2.sailor_id 
            AND r2.boat_id=b2.boat_id
            AND b2.color='green')

Can you change the above query to return the sailor's name instead of the sailor_id?

# Division in SQL

Here is the "easy" way:

In [ ]:
%%sql 

SELECT s.sailor_name
FROM sailors s
WHERE NOT EXISTS (
    SELECT b.boat_id
    FROM boats b
    EXCEPT 
        SELECT r.boat_id
        FROM reservations r
        WHERE r.sailor_id=s.sailor_id) ;

In [ ]:
%%sql 

SELECT s.sailor_name
FROM sailors s
WHERE NOT EXISTS(
    SELECT b.boat_id
    FROM boats b
    WHERE NOT EXISTS (
        SELECT r.boat_id
        FROM reservations r
        WHERE r.boat_id=b.boat_id
            AND r.sailor_id=s.sailor_id));

In [ ]:
%sql SELECT * FROM SAILORS;

In [ ]:
%sql SELECT sailor_name FROM sailors;

In [ ]:
%sql SELECT * FROM sailors WHERE sailor_name="Dustin";

In [ ]:
%sql SELECT sailor_name FROM sailors WHERE sailor_name="Dustin";

# Aggregate Operators

Now let's consider the following aggregate operators:
* COUNT (*)
* COUNT([DISTINCT] A)
* SUM ( [DISTINCT] A)
* AVG ( [DISTINCT] A)
* MAX (A)
* MIN (A)

In [1]:
%sql SELECT COUNT(*) FROM Sailors S;

UsageError: Line magic function `%sql` not found.
